In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import itertools

# Classe da rede neural CILP
class CILPNetwork:
    def __init__(self, num_entradas, num_regras, num_saidas):
        """
        Inicializa a rede neural com número de entradas, regras e saídas.
        """
        self.num_entradas = num_entradas
        self.num_regras = num_regras
        self.num_saidas = num_saidas
        self.pesos_entrada_oculta = np.zeros((num_regras, num_entradas))
        self.pesos_oculta_saida = np.zeros((num_saidas, num_regras))
        self.bias_oculta = np.zeros(num_regras)
        self.bias_saida = np.zeros(num_saidas)
        self.saidas = None  # Inicializa aqui

    def ativar(self, entrada):
        """
        Ativa a rede neural, calculando a saída com base nas entradas.
        """
        # Camada oculta
        soma_oculta = np.dot(self.pesos_entrada_oculta, entrada) + self.bias_oculta
        saida_oculta = np.sign(soma_oculta)
        saida_oculta[saida_oculta == 0] = -1  # Ajuste para considerar zero como -1

        # Camada de saída
        soma_saida = np.dot(self.pesos_oculta_saida, saida_oculta) + self.bias_saida
        saida = np.sign(soma_saida)
        saida[saida == 0] = -1  # Ajuste para zero virar -1

        return saida

def parse_regra(regra_str):
    """
    Parse uma regra no formato 'B, C, ~D => A' para um formato estruturado.
    Ajuste para regras que não possuem consequente.
    """
    if '=>' not in regra_str:
        return None
    antecedente_str, consequente_str = regra_str.split('=>')
    antecedente_str = antecedente_str.strip()
    consequente_str = consequente_str.strip()

    # Pega as variáveis do antecedente, permitindo ~ (negação)
    # Separa por ','
    antecedente = []
    for termo in antecedente_str.split(','):
        termo = termo.strip()
        if termo.startswith('~'):
            antecedente.append(('~', termo[1:]))
        else:
            antecedente.append(('+', termo))
    
    # Consequente: assume que só tem uma variável, possivelmente com negação
    if consequente_str:
        if consequente_str.startswith('~'):
            consequente = ('~', consequente_str[1:])
        else:
            consequente = ('+', consequente_str)
    else:
        # Caso o consequente seja vazio, não faz nada (ignora)
        return None

    return antecedente, consequente

def extrair_variaveis(regra_list):
    """
    Extrai todas as variáveis das regras para determinar as entradas da rede.
    """
    vars_ = set()
    for reg in regra_list:
        if reg is None:
            continue
        ant, cons = reg
        for sinal, var in ant:
            vars_.add(var)
        vars_.add(cons[1])
    return sorted(list(vars_))

def construir_rede(regra_strs, variaveis, saidas):
    """
    Constrói a rede neural com base nas regras lidas.
    """
    regras = [parse_regra(r) for r in regra_strs]
    # Remove regras inválidas (None)
    regras = [r for r in regras if r is not None]
    
    num_entradas = len(variaveis)
    num_regras = len(regras)
    num_saidas = len(saidas)
    rede = CILPNetwork(num_entradas, num_regras, num_saidas)

    # Mapeamento variável -> índice
    var_idx = {v: i for i, v in enumerate(variaveis)}
    saida_idx = {v: i for i, v in enumerate(saidas)}

    # Construir pesos da camada entrada -> oculta (uma linha por regra)
    for j, (antecedente, consequente) in enumerate(regras):
        for sinal, var in antecedente:
            i = var_idx[var]
            rede.pesos_entrada_oculta[j, i] = 1 if sinal == '+' else -1
        # Bias da camada oculta para regra j é - (tamanho do antecedente - 0.5)
        rede.bias_oculta[j] = -(len(antecedente) - 0.5)

    # Construir pesos da camada oculta -> saída
    for j, (antecedente, consequente) in enumerate(regras):
        sinal, var = consequente
        i = saida_idx[var]
        rede.pesos_oculta_saida[i, j] = 1 if sinal == '+' else -1

    # Bias da camada saída
    for i in range(num_saidas):
        rede.bias_saida[i] = -0.5

    rede.saidas = list(saidas)

    return rede

def gerar_tabela_testes(variaveis):
    """
    Gera todas as combinações possíveis de entradas para a tabela de testes.
    """
    n = len(variaveis)
    combinacoes = list(itertools.product([1, -1], repeat=n))
    return combinacoes

def testar_rede(rede, tabela, variaveis, saidas):
    """
    Testa a rede com a tabela de combinações de entradas e salva os resultados em um CSV.
    """
    # Lista para armazenar as entradas e saídas para exportação
    resultados = []

    for entrada in tabela:
        saida = rede.ativar(np.array(entrada))
        entrada_str = ", ".join(f"{v}={val}" for v, val in zip(variaveis, entrada))
        saida_str = ", ".join(f"{s}={int(o)}" for s, o in zip(saidas, saida))
        
        # Armazenar os resultados
        resultado = list(entrada) + list(saida)
        resultados.append(resultado)

    # Criar um DataFrame e salvar em CSV
    colunas = variaveis + saidas
    df_resultado = pd.DataFrame(resultados, columns=colunas)
    df_resultado.to_csv('resultados_rede.csv', index=False)

    print(f"Resultados salvos em 'resultados_rede.csv'.")

# Ler arquivo com as regras
df = pd.read_csv('../atividade-CILP/regras.csv')  # Substitua pelo caminho correto
regras_str = df['Regras'].tolist()

# Parse para extrair as variáveis e saídas
regras_parsed = [parse_regra(r) for r in regras_str]
# Filtra as regras válidas (não nulas)
regras_parsed = [r for r in regras_parsed if r is not None]

# Extrai as variáveis únicas
variaveis = extrair_variaveis(regras_parsed)

# Saídas: todas as variáveis que aparecem na cabeça das regras
saidas = sorted(set(cons[1] for _, cons in regras_parsed))

# Construir a rede
rede = construir_rede(regras_str, variaveis, saidas)

# Gerar tabela de testes com todas as combinações de entradas
tabela = gerar_tabela_testes(variaveis)

# Testar e salvar os resultados em CSV
testar_rede(rede, tabela, variaveis, saidas)


Resultados salvos em 'resultados_rede.csv'.
